In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50

## Generate Model 

In [5]:
cnn_model = models.Sequential([
    layers.Conv2D(input_shape=(128, 128, 3), 
                kernel_size = (3, 3), padding='same',
                filters=32,
                activation='relu'),
    layers.MaxPooling2D((2, 2), strides=2),

    layers.Conv2D(kernel_size = (3, 3), padding='same',
                filters=64,
                activation='relu'),
    layers.MaxPooling2D((2, 2), strides=2),

    layers.Conv2D(kernel_size = (3, 3), padding='same',
                filters=128,
                activation='relu'),
    layers.MaxPooling2D((2, 2), strides=2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 128)      0

In [6]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)
# base_model.trainable = False

resnet_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    # layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

resnet_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
Total params: 23,850,629
Trainable params: 23,797,509
Non-trainable params: 53,120
_________________________________________________________________


## Data Process

In [7]:
train_dir = "data/train"
test_dir = "data/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),       # ResNet50 input size
    batch_size=32,
    class_mode="categorical",
    subset="training"             # ← training split
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="validation"           # ← validation split
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,              # ← 라벨 없음
    shuffle=False                 # 결과 순서 유지
)

Found 2198 images belonging to 5 classes.
Found 548 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


## CNN

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

cnn_history = cnn_model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15
69/69 [==============================] - 24s 332ms/step - loss: 1.4102 - accuracy: 0.3763 - val_loss: 1.2732 - val_accuracy: 0.4599
Epoch 2/15
69/69 [==============================] - 23s 338ms/step - loss: 1.1372 - accuracy: 0.5209 - val_loss: 1.1692 - val_accuracy: 0.4982
Epoch 3/15
69/69 [==============================] - 23s 339ms/step - loss: 1.0230 - accuracy: 0.5946 - val_loss: 1.1414 - val_accuracy: 0.5511
Epoch 4/15
69/69 [==============================] - 24s 342ms/step - loss: 0.9751 - accuracy: 0.6015 - val_loss: 1.0257 - val_accuracy: 0.5894
Epoch 5/15
69/69 [==============================] - ETA: 0s - loss: 0.8811 - accuracy: 0.6611

## ResNet

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

resnet_history = resnet_model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

NameError: name 'resnet_model' is not defined

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("Detected GPU(s):", gpus)
else:
    print("No GPU detected, training will run on CPU.")

No GPU detected, training will run on CPU.


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_Oct_11_22:11:21_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.4, V11.4.152
Build cuda_11.4.r11.4/compiler.30521435_0
